In [ ]:
! pip install transformers==2.8.0

In [ ]:
! pip install pytorch-lightning

In [ ]:
! pip install pytorch-nlp

In [ ]:
! pip install paho-mqtt==1.5.0

In [ ]:
import pytorch_lightning as pl
import torch
from typing import List
from tqdm import tqdm
import re
import numpy as np
from argparse import Namespace
import logging

from transformers import ElectraTokenizer, ElectraModel

import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer, LayerNorm
from torchnlp.metrics import get_accuracy, get_token_accuracy
from pytorch_lightning.metrics.functional import f1_score
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau


import logging
from pprint import pformat
from itertools import chain
import torch
import torch.nn.functional as F
from transformers import AutoModelWithLMHead, AutoTokenizer

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/Chatbot/runs/DialoGPT")
model = AutoModelWithLMHead.from_pretrained("/content/drive/My Drive/Chatbot/runs/DialoGPT")
model.to(device)

In [ ]:
SPECIAL_TOKENS = ["<bos>", "<eos>", "<speaker1>", "<speaker2>", "[cls]", "<pad>"]
ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'cls_token': '[CLS]', 'pad_token': '<pad>', 
                         'additional_special_tokens': ['<speaker1>', '<speaker2>', 'anythingElse', 'none', 'greet', 'nothingElse', 'listProjects', 'helpYou', 'listTasks', 'thanks', 'onIt'],
                        }

orig_num_tokens = len(tokenizer.encoder)
num_added_tokens = tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN) # doesn't add if they are already there
if num_added_tokens > 0:
    model.resize_token_embeddings(new_num_tokens=orig_num_tokens + num_added_tokens)

In [ ]:
class RasaIntentEntityDataset(torch.utils.data.Dataset):

    def __init__(self, markdown_lines: List[str], tokenizer, seq_len=128,):
        self.intent_dict = {}
        self.entity_dict = {}
        self.entity_dict["O"] = 0  # using BIO tagging

        self.dataset = []
        self.seq_len = seq_len

        intent_value_list = []
        entity_type_list = []

        current_intent_focus = ""

        text_list = []

        for line in tqdm(markdown_lines,desc="Organizing Intent & Entity dictionary in NLU markdown file ...",):
            if len(line.strip()) < 2:
                current_intent_focus = ""
                continue

            if "## " in line:
                if "intent:" in line:
                    intent_value_list.append(line.split(":")[1].strip())
                    current_intent_focus = line.split(":")[1].strip()
                else:
                    current_intent_focus = ""

            else:
                if current_intent_focus != "":
                    text = line[2:].strip().lower()

                    for type_str in re.finditer(r"\([a-zA-Z_1-2]+\)", text):
                        entity_type = (text[type_str.start() + 1 : type_str.end() - 1].replace("(", "").replace(")", ""))
                        entity_type_list.append(entity_type)

                    text = re.sub(r"\([a-zA-Z_1-2]+\)", "", text)  # remove (...) str
                    text = text.replace("[", "").replace("]", "")  # remove '[',']' special char

                    if len(text) > 0:
                        text_list.append(text.strip())


        #dataset tokenizer setting
        if "ElectraTokenizer" in str(type(tokenizer)):
            self.tokenizer = tokenizer
            self.pad_token_id = 0
            self.unk_token_id = 1
            self.eos_token_id = 3 #[SEP] token
            self.bos_token_id = 2 #[CLS] token

        else:
            raise ValueError('not supported tokenizer type')

        intent_value_list = sorted(intent_value_list)
        for intent_value in intent_value_list:
            if intent_value not in self.intent_dict.keys():
                self.intent_dict[intent_value] = len(self.intent_dict)

        entity_type_list = sorted(entity_type_list)
        for entity_type in entity_type_list:
            if entity_type + '_B' not in self.entity_dict.keys():
                self.entity_dict[str(entity_type) + '_B'] = len(self.entity_dict)
            if entity_type + '_I' not in self.entity_dict.keys():
                self.entity_dict[str(entity_type) + '_I'] = len(self.entity_dict)

        current_intent_focus = ""

        for line in tqdm(markdown_lines, desc="Extracting Intent & Entity in NLU markdown files...",):
            if len(line.strip()) < 2:
                current_intent_focus = ""
                continue

            if "## " in line:
                if "intent:" in line:
                    current_intent_focus = line.split(":")[1].strip()
                else:
                    current_intent_focus = ""
            else:
                if current_intent_focus != "":  # intent & entity sentence occur case
                    text = line[2:].strip().lower()

                    entity_value_list = []
                    for value in re.finditer(r"\[(.*?)\]", text):
                        entity_value_list.append(text[value.start() + 1 : value.end() - 1].replace("[","").replace("]",""))

                    entity_type_list = []
                    for type_str in re.finditer(r"\([a-zA-Z_1-2]+\)", text):
                        entity_type = (text[type_str.start() + 1 : type_str.end() - 1].replace("(","").replace(")",""))
                        entity_type_list.append(entity_type)

                    text = re.sub(r"\([a-zA-Z_1-2]+\)", "", text)  # remove (...) str
                    text = text.replace("[", "").replace("]", "")  # remove '[',']' special char

                    if len(text) > 0:
                        each_data_dict = {}
                        each_data_dict["text"] = text.strip()
                        each_data_dict["intent"] = current_intent_focus
                        each_data_dict["intent_idx"] = self.intent_dict[current_intent_focus]
                        each_data_dict["entities"] = []

                        for value, type_str in zip(entity_value_list, entity_type_list):
                            for entity in re.finditer(value, text):
                                entity_tokens = self.tokenize(value)

                                for i, entity_token in enumerate(entity_tokens):
                                    if i == 0:
                                        BIO_type_str = type_str + '_B'
                                    else:
                                        BIO_type_str = type_str + '_I'

                                    each_data_dict["entities"].append(
                                        {
                                            "start": text.find(entity_token, entity.start(), entity.end()),
                                            "end": text.find(entity_token, entity.start(), entity.end()) + len(entity_token),
                                            "entity": type_str,
                                            "value": entity_token,
                                            "entity_idx": self.entity_dict[BIO_type_str],
                                        }
                                    )


                        self.dataset.append(each_data_dict)

        
        print(f"Intents: {self.intent_dict}")
        print(f"Entities: {self.entity_dict}")

    def tokenize(self, text: str, skip_special_char=True):
        if "ElectraTokenizer" in str(type(self.tokenizer)):
            if skip_special_char:
                return self.tokenizer.tokenize(text)
            else:
                return [token.replace('#','') for token in self.tokenizer.tokenize(text)]
        else:
            raise ValueError('not supported tokenizer type')
            
    def encode(self, text: str, padding: bool = True, return_tensor: bool = True):
        tokens = self.tokenizer.encode(text)
        if type(tokens) == list:
            tokens = torch.tensor(tokens).long()
        else:
            tokens = tokens.long()

        if padding:
            if len(tokens) >= self.seq_len:
                tokens = tokens[: self.seq_len]
            else:
                pad_tensor = torch.tensor([self.pad_token_id] * (self.seq_len - len(tokens)))
            
                tokens = torch.cat((tokens, pad_tensor), 0)

        if return_tensor:
            return tokens
        else:
            return tokens.numpy()

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        tokens = self.encode(self.dataset[idx]["text"])

        intent_idx = torch.tensor([self.dataset[idx]["intent_idx"]])

        entity_idx = np.array(self.seq_len * [0]) # O tag indicate 0(zero)

        for entity_info in self.dataset[idx]["entities"]:
            if "ElectraTokenizer" in str(type(self.tokenizer)):
                ##check whether entity value is include in splitted token
                for token_seq, token_value in enumerate(tokens):
                    # Consider [CLS](bos) token
                    if token_seq == 0:
                        continue

                    for entity_seq, entity_info in enumerate(self.dataset[idx]["entities"]):
                        if (self.tokenizer.convert_ids_to_tokens([token_value.item()])[0] in entity_info["value"]):
                            entity_idx[token_seq] = entity_info["entity_idx"]
                            break

        entity_idx = torch.from_numpy(entity_idx)

        return tokens, intent_idx, entity_idx, self.dataset[idx]["text"]

    def get_intent_idx(self):
        return self.intent_dict

    def get_entity_idx(self):
        return self.entity_dict

    def get_vocab_size(self):
        return self.tokenizer.vocab_size

    def get_seq_len(self):
        return self.seq_len

In [ ]:
class EmbeddingTransformer(nn.Module):
    def __init__(
        self,
        backbone: None,
        vocab_size: int,
        seq_len: int,
        intent_class_num: int,
        entity_class_num: int,
        d_model=512,
        nhead=8,
        num_encoder_layers=6,
        dim_feedforward=2048,
        dropout=0.1,
        activation="relu",
        pad_token_id: int = 0,
    ):
        super(EmbeddingTransformer, self).__init__()
        self.backbone = backbone
        self.seq_len = seq_len
        self.pad_token_id = pad_token_id

        if backbone is None:
            self.encoder = nn.TransformerEncoder(
                TransformerEncoderLayer(
                    d_model, nhead, dim_feedforward, dropout, activation
                ),
                num_encoder_layers,
                LayerNorm(d_model),
            )
        else:  # pre-defined model architecture use
            if backbone == "electra":
                self.encoder = ElectraModel.from_pretrained("google/electra-small-discriminator")

            d_model = self.encoder.config.hidden_size

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(self.seq_len, d_model)

        self.intent_feature = nn.Linear(d_model, intent_class_num)
        self.entity_feature = nn.Linear(d_model, entity_class_num)

    def forward(self, x):
        if self.backbone in ["electra"]:
            feature = self.encoder(x)

            if type(feature) == tuple:
                feature = feature[0]  # last_hidden_state (N,S,E)

            # first token in sequence used to intent classification
            intent_feature = self.intent_feature(feature[:, 0, :]) # (N,E) -> (N,i_C)

            # other tokens in sequence used to entity classification
            entity_feature = self.entity_feature(feature[:, :, :]) # (N,S,E) -> (N,S,e_C)

            return intent_feature, entity_feature

In [ ]:
class DualIntentEntityTransformer(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()

        self.hparams = hparams
        if type(self.hparams) == dict:
            self.hparams = Namespace(**self.hparams)

        self.dataset = RasaIntentEntityDataset(
            markdown_lines=self.hparams.nlu_data,
            tokenizer=self.hparams.tokenizer,
        )

        self.model = EmbeddingTransformer(
            backbone=self.hparams.backbone,
            vocab_size=self.dataset.get_vocab_size(),
            seq_len=self.dataset.get_seq_len(),
            intent_class_num=len(self.dataset.get_intent_idx()),
            entity_class_num=len(self.dataset.get_entity_idx()),
            d_model=self.hparams.d_model,
            num_encoder_layers=self.hparams.num_encoder_layers,
            pad_token_id=self.dataset.pad_token_id
        )

        self.train_ratio = self.hparams.train_ratio
        self.batch_size = self.hparams.batch_size
        self.optimizer = self.hparams.optimizer
        self.intent_optimizer_lr = self.hparams.intent_optimizer_lr
        self.entity_optimizer_lr = self.hparams.entity_optimizer_lr

        self.intent_loss_fn = nn.CrossEntropyLoss()
        # reduce O tag class weight to figure out entity imbalance distribution
        self.entity_loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor([0.1] + [1.0] * (len(self.dataset.get_entity_idx()) - 1)))

    def forward(self, x):
        return self.model(x)

    def prepare_data(self):
        train_length = int(len(self.dataset) * self.train_ratio)

        self.train_dataset, self.val_dataset = random_split(
            self.dataset, [train_length, len(self.dataset) - train_length],
        )

        self.hparams.intent_label = self.get_intent_label()
        self.hparams.entity_label = self.get_entity_label()
    
    def get_intent_label(self):
        self.intent_dict = {}
        for k, v in self.dataset.intent_dict.items():
            self.intent_dict[str(v)] = k
        return self.intent_dict 
    
    def get_entity_label(self):
        self.entity_dict = {}
        for k, v in self.dataset.entity_dict.items():
            self.entity_dict[str(v)] = k
        return self.entity_dict

    def train_dataloader(self):
        train_loader = DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
        )
        return train_loader

    def val_dataloader(self):
        val_loader = DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
        )
        return val_loader

    def configure_optimizers(self):
        intent_optimizer = eval(
            f"{self.optimizer}(self.parameters(), lr={self.intent_optimizer_lr})"
        )
        entity_optimizer = eval(
            f"{self.optimizer}(self.parameters(), lr={self.entity_optimizer_lr})"
        )

        return (
            [intent_optimizer, entity_optimizer],
            # [StepLR(intent_optimizer, step_size=1),StepLR(entity_optimizer, step_size=1),],
            [
                ReduceLROnPlateau(intent_optimizer, patience=1),
                ReduceLROnPlateau(entity_optimizer, patience=1),
            ],
        )

    def training_step(self, batch, batch_idx, optimizer_idx):
        self.model.train()

        tokens, intent_idx, entity_idx, text = batch
        intent_pred, entity_pred = self.forward(tokens)

        intent_acc = get_accuracy(intent_pred.argmax(1).cpu(), intent_idx.cpu())[0]
        #entity_acc = get_token_accuracy(entity_pred.argmax(2), entity_idx, ignore_index=self.dataset.pad_token_id)[0]

        tensorboard_logs = {
            "train/intent/acc": intent_acc,
            #"train/entity/acc": entity_acc,
        }

        if optimizer_idx == 0:
            intent_loss = self.intent_loss_fn(intent_pred, intent_idx.squeeze(1))
            tensorboard_logs["train/intent/loss"] = intent_loss

            return {
                "loss": intent_loss,
                "log": tensorboard_logs,
            }

        if optimizer_idx == 1:
            entity_loss = self.entity_loss_fn(entity_pred.transpose(1, 2), entity_idx.long(),)
            tensorboard_logs["train/entity/loss"] = entity_loss

            return {
                "loss": entity_loss,
                "log": tensorboard_logs,
            }

    def validation_step(self, batch, batch_idx):
        self.model.eval()

        tokens, intent_idx, entity_idx, text = batch
        intent_pred, entity_pred = self.forward(tokens)
        

        intent_acc = get_accuracy(intent_pred.argmax(1).cpu(), intent_idx.cpu())[0]
        #entity_acc = get_token_accuracy(entity_pred.argmax(2), entity_idx, ignore_index=self.dataset.pad_token_id)[0]
        intent_f1 = f1_score(intent_pred.argmax(1), intent_idx)

        intent_loss = self.intent_loss_fn(intent_pred, intent_idx.squeeze(1))
        entity_loss = self.entity_loss_fn(entity_pred.transpose(1, 2), entity_idx.long(),)

        return {
            "val_intent_acc": torch.Tensor([intent_acc]),
            #"val_entity_acc": torch.Tensor([entity_acc]),
            "val_intent_f1": intent_f1,
            "val_intent_loss": intent_loss,
            "val_entity_loss": entity_loss,
            "val_loss": intent_loss + entity_loss,
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_intent_acc = torch.stack([x["val_intent_acc"] for x in outputs]).mean()
        #avg_entity_acc = torch.stack([x["val_entity_acc"] for x in outputs]).mean()
        avg_intent_f1 = torch.stack([x["val_intent_f1"] for x in outputs]).mean()

        tensorboard_logs = {
            "val/loss": avg_loss,
            "val/intent_acc": avg_intent_acc,
            #"val/entity_acc": avg_entity_acc,
            "val/intent_f1": avg_intent_f1,
        }

        return {
            "val_loss": avg_loss,
            "val_intent_f1": avg_intent_f1,
            #"val_entity_acc": avg_entity_acc,
            "log": tensorboard_logs,
            "progress_bar": tensorboard_logs,
        }

In [ ]:
model = None
intent_dict = {}
entity_dict = {}

class Inferencer:
    def __init__(self, checkpoint_path: str):
        self.model = DualIntentEntityTransformer.load_from_checkpoint(checkpoint_path)
        self.model.model.eval()

        self.intent_dict = {}
        for k, v in self.model.dataset.intent_dict.items():
            self.intent_dict[v] = k # str key -> int key

        self.entity_dict = {}
        for k, v in self.model.dataset.entity_dict.items():
            self.entity_dict[v] = k # str key -> int key

        logging.info("intent dictionary")
        logging.info(self.intent_dict)
        print()

        logging.info("entity dictionary")
        logging.info(self.entity_dict)

    def is_same_entity(self, i, j):
        # check whether XXX_B, XXX_I tag are same 
        return self.entity_dict[i][:self.entity_dict[i].rfind('_')].strip() == self.entity_dict[j][:self.entity_dict[j].rfind('_')].strip()

    def inference(self, text: str, intent_topk=5):
        if self.model is None:
            raise ValueError(
                "model is not loaded, first call load_model(checkpoint_path)"
            )

        text = text.strip().lower()

        # encode text to token_indices
        tokens = self.model.dataset.encode(text)
        intent_result, entity_result = self.model.forward(tokens.unsqueeze(0).cpu())

        # mapping intent result
        rank_values, rank_indicies = torch.topk(
            nn.Softmax(dim=1)(intent_result)[0], k=intent_topk
        )
        intent = {}
        intent_ranking = []
        for i, (value, index) in enumerate(
            list(zip(rank_values.tolist(), rank_indicies.tolist()))
        ):
            intent_ranking.append(
                {"confidence": value, "name": self.intent_dict[index]}
            )

            if i == 0:
                intent["name"] = self.intent_dict[index]
                intent["confidence"] = value

        # mapping entity result
        entities = []

        # except first & last sequnce token whcih indicate BOS or [CLS] token & EOS or [SEP] token
        _, entity_indices = torch.max((entity_result)[0][1:-1, :], dim=1)
        start_idx = -1

        #print ('tokens')
        #print (tokens)
        #print ('predicted entities')
        #print (entity_indices)

        entity_indices = entity_indices.tolist()[:len(text)]
        start_token_position = -1

        # except first sequnce token whcih indicate BOS or [CLS] token
        if type(tokens) == torch.Tensor:
            tokens = tokens.long().tolist()

        for i, entity_idx_value in enumerate(entity_indices):
            if entity_idx_value != 0 and start_token_position == -1:
                start_token_position = i
            elif start_token_position >= 0 and not self.is_same_entity(entity_indices[i-1],entity_indices[i]):
                end_token_position = i - 1

                #print ('start_token_position')
                #print (start_token_position)
                #print ('end_token_position')
                #print (end_token_position)

                # find start text position
                token_idx = tokens[start_token_position + 1]
                if "ElectraTokenizer" in str(
                    type(self.model.dataset.tokenizer)
                ):  # ElectraTokenizer
                    token_value = self.model.dataset.tokenizer.convert_ids_to_tokens([token_idx])[0].replace("#", "")

                if len(token_value.strip()) == 0:
                    start_token_position = -1
                    continue
                  
                start_position = text.find(token_value.strip())

                # find end text position
                token_idx = tokens[end_token_position + 1]
                if "ElectraTokenizer" in str(
                    type(self.model.dataset.tokenizer)
                ):  # ElectraTokenizer
                    token_value = self.model.dataset.tokenizer.convert_ids_to_tokens([token_idx])[0].replace("#", "")

                end_position = text.find(token_value.strip(), start_position) + len(token_value.strip())

                if self.entity_dict[entity_indices[i-1]] != "O": # ignore 'O' tag
                    entities.append(
                         {
                            "start": start_position,
                            "end": end_position,
                            "value": text[start_position:end_position],
                            "entity": self.entity_dict[entity_indices[i-1]][:self.entity_dict[entity_indices[i-1]].rfind('_')]
                        }
                    )
                      
                    start_token_position = -1

                if entity_idx_value == 0:
                    start_token_position = -1

        result = {
            "text": text,
            "intent": intent,
            "intent_ranking": intent_ranking,
            "entities": entities,
        }

        # print (result)

        return result

In [ ]:
def build_inputs(history, reply, tokenizer, lm_labels=True, with_eos=True):
    """ Build a sequence of input from 3 segments: persona, history and last reply. """
    bos, eos, speaker1, speaker2 = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-2])
    sequence = [[bos]] + history + [reply + ([eos] if with_eos else [])]
    sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])]
    instance = {}
    instance["input_ids"] = list(chain(*sequence))
    instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
    #instance["mc_token_ids"] = len(instance["input_ids"]) - 1
    instance["lm_labels"] = [-100] * len(instance["input_ids"])
    if lm_labels:
        instance["lm_labels"] = ([-100] * sum(len(s) for s in sequence[:-1])) + [-100] + sequence[-1][1:]
    return instance

In [ ]:
def top_filtering(logits, top_k=0., top_p=0.9, threshold=-float('Inf'), filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k, top-p (nucleus) and/or threshold filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k: <=0: no filtering, >0: keep only top k tokens with highest probability.
            top_p: <=0.0: no filtering, >0.0: keep only a subset S of candidates, where S is the smallest subset
                whose total probability mass is greater than or equal to the threshold top_p.
                In practice, we select the highest probability tokens whose cumulative probability mass exceeds
                the threshold top_p.
            threshold: a minimal threshold to keep logits
    """
    assert logits.dim() == 1  # Only work for batch size 1 for now - could update but it would obfuscate a bit the code
    top_k = min(top_k, logits.size(-1))
    if top_k > 0:
        # Remove all tokens with a probability less than the last token in the top-k tokens
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        # Compute cumulative probabilities of sorted tokens
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probabilities = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probabilities > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # Back to unsorted indices and set them to -infinity
        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value

    indices_to_remove = logits < threshold
    logits[indices_to_remove] = filter_value

    return logits

In [ ]:
def sample_sequence(history, tokenizer, model, current_output=None):
    special_tokens_ids = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
    if current_output is None:
        current_output = []
    
    min_length = 1
    max_length = 20
    temperature = 0.7
    top_k = 0
    top_p = 0.9
    no_sample = False
    
    for i in range(max_length):
        instance = build_inputs(history, current_output, tokenizer, with_eos=False)

        input_ids = torch.tensor(instance["input_ids"], device=device).unsqueeze(0)
        token_type_ids = torch.tensor(instance["token_type_ids"], device=device).unsqueeze(0)

        logits = model(input_ids, token_type_ids=token_type_ids)
        if isinstance(logits, tuple):  # for gpt2 and maybe others
            logits = logits[0]
        logits = logits[0, -1, :] / temperature
        logits = top_filtering(logits, top_k=top_k, top_p=top_p)
        probs = F.softmax(logits, dim=-1)

        prev = torch.topk(probs, 1)[1] if no_sample else torch.multinomial(probs, 1)
        if i < min_length and prev.item() in special_tokens_ids:
            while prev.item() in special_tokens_ids:
                if probs.max().item() == 1:
                    warnings.warn("Warning: model generating special token with probability 1.")
                    break  # avoid infinitely looping over special token
                prev = torch.multinomial(probs, num_samples=1)

        if prev.item() in special_tokens_ids:
            break
        current_output.append(prev.item())

    return current_output

In [ ]:
import paho.mqtt.client as mqtt
import time
import json

broker_address = "test.mosquitto.org"
port = 1883

In [ ]:
inferencer = Inferencer("/content/drive/My Drive/IntentEntity/lightning_logs/version_0/checkpoints/epoch=19.ckpt")

In [ ]:
#history = []


def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    #client.subscribe("testChat/question")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, message):
    #print(message.topic+" "+str(message.payload))
    global FLAG
    global chat
    history = []
    if str(message.topic) != publishTopic:
        user_input = str(message.payload.decode("utf-8"))
        print('user_input: ', user_input)
        #print(str(message.topic), user_input)
        if user_input == "quit":
            FLAG = False
        else:
            print('1')
            history.append(tokenizer.encode(user_input))
            print('2')
            with torch.no_grad():
                print('3')
                out_ids = sample_sequence(history, tokenizer, model)
                print('4')
            print('5')
            history.append(out_ids)
            print(history)
            
            intent_entity = inferencer.inference(user_input, intent_topk=5)
            intent = intent_entity['intent']['name']
            entities = []
        
            if len(intent_entity['entities'])>0:
                for i in range(len(intent_entity['entities'])):
                    entities[i] = {'entity': intent_entity['entities'][i]['entity'], 'value':intent_entity['entities'][i]['value']}
                    print(entities[i])
            history = history[-3:]
            out_text = tokenizer.decode(out_ids, skip_special_tokens=True)
            
            data = {'intent': intent, 'entity': entities, 'reply': out_text}

            client.publish(publishTopic, out_text)
        

def on_subscribe(client, userdata, mid, granted_qos):
    print("Subscribed: ", str(mid), str(granted_qos))

def on_unsubscribe(client, userdata, mid):
    print("Unsubscribed: ", str(mid))
    
def on_disconnect(client, userdata, rc):
    if (rc != 0):
        print("Unexpected Disconnection")


In [ ]:
client = mqtt.Client()
client.on_subscribe = on_subscribe
client.on_unsubscribe = on_unsubscribe
client.on_connect = on_connect
client.on_message = on_message
client.connect(broker_address, port)

time.sleep(1)

publishTopic = 'Chat/answer'
subscribeTopic = 'Chat/question'

FLAG = True

client.loop_start()
client.subscribe(subscribeTopic, qos=2)

time.sleep(1)

chat = input("Enter Message: ")
client.publish(publishTopic, chat, qos=2)

while True:
    if FLAG == False or chat == "quit":
        break
        
client.disconnect()
client.loop_stop()

In [ ]:
"""def run():
    inferencer = Inferencer("lightning_logs/version_35/checkpoints/epoch=19.ckpt")
    
    history = []
    while True:
        user_input = input(">>> ")
        while not user_input:
            print('Prompt should not be empty!')
            user_input = input(">>> ")
        if user_input == 'quit':
            break
        history.append(tokenizer.encode(user_input))
        with torch.no_grad():
            out_ids = sample_sequence(history, tokenizer, model)
        history.append(out_ids)
        
        intent_entity = inferencer.inference(user_input, intent_topk=5)
        intent = intent_entity['intent']['name']
        entities = []
        
        if len(intent_entity['entities'])>0:
            for i in range(len(intent_entity['entities'])):
                entities[i] = {'entity': intent_entity['entities'][i]['entity'], 'value':intent_entity['entities'][i]['value']}
                print(entities[i])
        
        #history = history[-(2*2+1):]
        ######Only the last 3
        history = history[-3:]
        out_text = tokenizer.decode(out_ids, skip_special_tokens=True)
        print(out_text)"""

In [ ]:
#run()